In [1]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
xpos=np.random.uniform(-10,10,100)
ypos=np.random.uniform(-10,10,100)
zpos=np.random.uniform(-10,10,100)

Aca definimos los parámetros de la mesh

Para este caso, tenemos que las longitudes en las que se encuentran las partículas  son $L_x=20$, $L_y=20$, $L_z=20$ ya que $|-10-10|=20$

Vamos a tomar las longitudes de los largos de las celdas como:
$\Delta x= \frac{L_x}{N_{g_x}}$,
$\Delta y= \frac{L_y}{N_{g_y}}$,
$\Delta z= \frac{L_z}{N_{g_z}}$

In [23]:
Lx=20.0
Ly=20.0
Lz=20.0
N=20

In [24]:
Deltax=(Lx/N)
Deltay=(Ly/N)
Deltaz=(Lz/N)

Para calcular la densidad, primero necesitamos un arreglo de tamaño  $N_{gx}\times N_{gy}\times N_{gz}$, para nuestro caso es $N^3$. Definamos esa variable:

In [27]:
den3D=np.zeros((N,N,N))

La función forma de la partícula en este caso es un paralelepipedo rectangular  de volume, $\Delta x \Delta y \Delta z$. En este esquema se involucran las ocho celdas más cercanas para cada partícula. Este esquema es conocido como Cloud-in-Cell (CiC).

Consideremos, el esquema de asignación CiC para una partícula con coordenadas $(x_p,y_p,z_p)$. La celda que contiene 
la partícula tiene índices dados por: $i=floor(x_p/\Delta x)$, $j=floor(y_p/\Delta y)$, $k=floor(z_p/\Delta z)$

Definamos una función que nos retorne estos valores:

In [56]:
def INDEX(x_p,y_p,z_p,Deltax,Deltay,Deltaz):
    i=math.floor(x_p/Deltax)
    j=math.floor(y_p/Deltay)
    k=math.floor(z_p/Deltaz)
    return np.array([i,j,k])

Debemos considerar el centro de la celda el cuál va estar definido por $(x_c,y_c,z_c)=(i\Delta x, j \Delta y, k\Delta z)$

In [65]:
def CenCelda(x_p,y_p,z_p,Deltax,Deltay,Deltaz):
    return np.array([INDEX(x_p,y_p,z_p,Deltax,Deltay,Deltax)[0]* Deltax,INDEX(x_p,y_p,z_p,Deltax,Deltay,Deltax)[1]* Deltay,INDEX(x_p,y_p,z_p,Deltax,Deltay,Deltax)[2]* Deltaz])

La densisdad contribuye a la celda principal y a siete celdas vecinas indexadas por $(i,j,k), (ii,j,k),(i,jj,k),(i,j,kk),(ii,jj,k),(ii,j,kk),(i,jj,kk)$ y $(ii,jj,kk)$

Donde los índices $ii$,$jj$ y $kk$ están definidos por: $ii=mod(i+1, N_{gx})$,$jj=mod(j+1, N_{gy})$ y 
$kk=mod(k+1, N_{gz})$

Definamos ahora las cantidades $dx=\frac{x_p}{\Delta x}-i$, $dy=\frac{y_p}{\Delta y}-j$, $dz=\frac{z_p}{\Delta z}-k$

y $t_x=1-dx$, $t_y=1-dy$, $t_z=1-dz$

Las contribuciones a las ocho celdas están dadas por $W(\vec{x_P}-x_{i,j,k})=t_x t_y t_z$

$W(\vec{x_P}-x_{ii,j,k})=d_x t_y t_z$

$W(\vec{x_P}-x_{i,jj,k})=t_x d_y t_z$

$W(\vec{x_P}-x_{i,j,kk})=t_x t_y d_z$

$W(\vec{x_P}-x_{ii,jj,k})=d_x d_y t_z$

$W(\vec{x_P}-x_{ii,j,kk})=d_x t_y d_z$

$W(\vec{x_P}-x_{i,jj,kk})=t_x d_y d_z$


$W(\vec{x_P}-x_{ii,jj,kk})=d_x d_y d_z$

Sumando sobre todas las partículas resulta en el calculo de $\rho$ en la grid. Pregunta: Sabemos que $\rho(x_c)=Cons\sum_{i}^{N_p}W(\vec{x_i}-\vec{x_{ijk}})$

En el anterior código lo hacía $\rho(x_c)=C\sum_n W(x_n-x_i)W(y_n-y_j)W(z_n-z_k)$